In [ ]:
# Source of optical flow code: https://www.geeksforgeeks.org/opencv-the-gunnar-farneback-optical-flow/

# Kalman flow code: Author: Ruchik Mishra

In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import random

In [ ]:
# The video feed is read in as
# a VideoCapture object
cap = cv.VideoCapture("test_video.mp4")

# ret = a boolean return value from
# getting the frame, first_frame = the
# first frame in the entire video sequence
ret, first_frame = cap.read()
  
# Converts frame to grayscale because we
# only need the luminance channel for
# detecting edges - less computationally 
# expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
  
# Creates an image filled with zero
# intensities with the same dimensions 
# as the frame
mask = np.zeros_like(first_frame)
  
# Sets image saturation to maximum
mask[..., 1] = 255

print(prev_gray.shape)
print(np.prod(prev_gray.shape))

In [ ]:
prev_gray_area = np.prod(prev_gray.shape)
store_optical_flow = np.empty((prev_gray_area,1))
store_pixel_position = np.empty((prev_gray_area,1))

while cap.isOpened():
    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video
    ret, frame = cap.read()
    
    if not ret:
        print('Video has finished playing')
        break
      
    # Opens a new window and displays the input
    # frame
    cv.imshow("input", frame)
      
    # Converts each frame to grayscale - we previously 
    # only converted the first frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # tracking the time between the calculation of optical flow for the frames
    # start = time.time()
    
    # Calculates dense optical flow by Farneback method
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, 
                                       None,
                                       0.5, 3, 15, 3, 5, 1.2, 0)
    store_optical_flow = np.concatenate((store_optical_flow, np.reshape(flow[:,:,1], (prev_gray_area,1))), axis=1)

    # print(np.reshape(flow[:,:,1],(np.prod(prev_gray.shape),1)))
    # print(store_optical_flow.shape)
    # end = time.time()
    # print(end-start)

    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
      
    # Sets image hue according to the optical flow 
    # direction
    mask[..., 0] = angle * 180 / np.pi / 2
      
    # Sets image value according to the optical flow
    # magnitude (normalized)
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
      
    # print(mask.shape)
    
    # Converts HSV to RGB (BGR) color representation
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    # Opens a new window and displays the output frame
    cv.imshow("dense optical flow", rgb)
    
    # Convert to grayscale
    gray_scale_image = cv.cvtColor(mask, cv.COLOR_RGB2GRAY).reshape(prev_gray_area, 1)
    store_pixel_position = np.concatenate((store_pixel_position, gray_scale_image), axis=1)
    # print(store_pixel_position.shape)
  
    # Updates previous frame
    prev_gray = gray
    prev_gray_area = np.prod(prev_gray.shape)
      
    # Frames are read by intervals of 1 millisecond. The
    # programs breaks out of the while loop when the
    # user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# The following frees up resources and
# closes all windows
cap.release()
cv.destroyAllWindows()

In [ ]:
# both of these are (921600, 150) at end of video
print(store_optical_flow.shape)
print(store_pixel_position.shape)

# sort_index= np.argsort(np.cov(store_optical_flow,axis=1))

# print(np.argmax(store_optical_flow,1))
result = np.where(store_optical_flow == np.amax(store_optical_flow))
print(result)
print(store_optical_flow[324409,42])
print(store_optical_flow[324410,42])
print(store_optical_flow.shape[1])

In [ ]:
## Implementing the Kalman filter

dt = 1/30
A = np.array([[1,dt],[0,1]])
Q = np.diag(np.array([0.05,0.05]))
H = np.array([[1,0],[0,1]])

P_initial = np.eye(2)
x_initial =np.array([[random.random()],[0]])
R = np.diag(np.array([6,6]))

x_k_minus = np.array([[random.random()],[0]])
P_k_minus = np.eye(x_k_minus.shape[0])

print(dt)
print(A)
print(Q)
print("This is H: ", H)
print(P_initial)
print(x_initial)
print(R)
print('store_optical_flow shape', store_optical_flow.shape)
print('store_pixel_position shape', store_pixel_position.shape)
print("Checking multiplication using @ symbol: ", A@Q)
print(x_k_minus)
print(P_k_minus)
print(x_k_minus.shape)

In [ ]:
# Time update equations

# To prevent all the pixels to go through the filter to reduce run time
row_count = store_optical_flow.shape[0] - 921000
col_count = store_optical_flow.shape[1] 

# FULL SIZE MATRIX MIGHT TAKE UP 6GB IN MEMORY, DO NOT RUN FULL SIZE
store_predicted_states = np.empty((2 * row_count, col_count))

# i in range(600)
for i in range(row_count):
    # j in range(105)
    for j in range(col_count):
        # z.shape = (2,1)
        z = np.array([[store_pixel_position[i,j]],[store_optical_flow[i,j]]])
        
        print("Pixel position: ",store_pixel_position[i,j])
        print("Optical flow: ",store_optical_flow[i,j])
        
        # K.shape = (2,2)
        K = P_k_minus @ H.T@np.linalg.inv(H@P_k_minus@H.T +R)
        # x_pred_k.shape = (2,1)
        x_pred_k = x_k_minus + K@(z-H@x_k_minus)
        
        # Store the prediction in column j across rows i and i+1
        store_predicted_states[i:i+2, j] = x_pred_k.reshape(2,)
        
        # Updating stuff for next iter
        # P_k.shape = (2,2)
        P_k = (np.eye(x_k_minus.shape[0])-K@H)@P_k_minus
        # x_k_minus.shape = (2,1)
        x_k_minus = A@x_pred_k
        # P_k_minus.shape = (2,2)
        P_k_minus = A@P_k@A.T +Q

# should be (2*row_count, col_count) i.e. (1200, 105)
# print(store_predicted_states.shape)
#     store_for_each_pixel = np.concatenate((store_for_each_pixel,store_predicted_states),axis =0)
#     store_predicted_states =np.empty([2,2])

print(store_predicted_states[40,:])
print()
plt.plot(store_predicted_states[40,:])
plt.plot(store_pixel_position[20,0:col_count])
plt.xlabel("Number of frames")
plt.ylabel("X position")
plt.title("Comparing results of position estimation with and without Kalman Filter")
plt.legend(['KF','no KF'])
plt.show()